# Исследование уровня потребительской лояльности телекоммуникационной компании

Цель исследования - определить текущий уровень потребительской лояльности (NPS) среди клиентов из России.
К задачам исследования можно отнести ответы на следующие вопросы:
- Как распределены участники опроса по возрасту и полу? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?
- Какие группы пользователей наиболее лояльны к сервису? Какие менее?
- Какой общий NPS среди всех опрошенных?
- Как можно описать клиентов, которые относятся к группе cторонников (англ. promoters)?

In [1]:
# импорт библиотек и функций
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
# подключаемся к БД
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

Предварительно увидев, что в поле `age` имеются пропуски, с помощью оператора `WHERE` исключим их. 0,11% удаленных строк - это незначительный процент, который не повлияет на дальнейшее исследование.

In [3]:
# формируем sql-запрос, который выгрузит нужные поля
query = """
SELECT u.user_id,
       u.lt_day,
       CASE 
           WHEN u.lt_day <= 365 THEN 'Новый'
           ELSE 'Существующий'
           END AS is_new,
       u.age,
       CASE 
           WHEN u.gender_segment == 1 THEN 'Женщина'
           WHEN u.gender_segment == 0 THEN 'Мужчина'
           ELSE 'Не указан'
           END AS gender_segment,
       u.os_name,
       u.cpe_type_name,
       l.country,
       l.city,
       SUBSTR(age.title, 4) AS age_segment,
       SUBSTR(t.title, 4) AS traffic_segment,
       SUBSTR(lt.title, 4) AS lifetime_segment,
       u.nps_score,
       CASE 
           WHEN u.nps_score >= 9 THEN 'Cторонник'
           WHEN u.nps_score >= 7 THEN 'Нейтрал'
           ELSE 'Критик'
           END AS nps_group
FROM user AS u
LEFT JOIN location AS l ON u.location_id=l.location_id
LEFT JOIN age_segment AS age ON u.age_gr_id=age.age_gr_id
LEFT JOIN traffic_segment AS t ON u.tr_gr_id=t.tr_gr_id
LEFT JOIN lifetime_segment AS lt ON u.lt_gr_id=lt.lt_gr_id
WHERE u.age IS NOT NULL;
"""

In [4]:
# выполняем запрос, сохраняем результат выполнения в DataFrame и выводим первые 5 строк на экран
df = pd.read_sql(query, engine)
df.head(5)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,Существующий,45.0,Женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,Cторонник
1,A001WF,2344,Существующий,53.0,Мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,Cторонник
2,A003Q7,467,Существующий,57.0,Мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,Cторонник
3,A004TB,4190,Существующий,44.0,Женщина,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,Cторонник
4,A004XT,1163,Существующий,24.0,Мужчина,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,Cторонник


In [5]:
# изучим DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501939 entries, 0 to 501938
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           501939 non-null  object 
 1   lt_day            501939 non-null  int64  
 2   is_new            501939 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    501939 non-null  object 
 5   os_name           501939 non-null  object 
 6   cpe_type_name     501939 non-null  object 
 7   country           501939 non-null  object 
 8   city              501939 non-null  object 
 9   age_segment       501939 non-null  object 
 10  traffic_segment   501939 non-null  object 
 11  lifetime_segment  501939 non-null  object 
 12  nps_score         501939 non-null  int64  
 13  nps_group         501939 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.6+ MB


В таблице 14 столбцов с данными разных типов:

* `user_id` — идентификатор клиента;
* `lt_day` — количество дней «жизни» клиента;
* `is_new` — поле хранит информацию о том, является ли клиент новым;
* `age` — возраст;
* `gender_segment` — пол;
* `os_name` — тип операционной системы;
* `cpe_type_name` — тип устройства;
* `country` — страна проживания;
* `city` — город проживания;
* `age_segment` — возрастной сегмент;
* `traffic_segment` — сегмент по объёму потребляемого трафика;
* `lifetime_segment` — сегмент по количеству дней «жизни»;
* `nps_score` — оценка клиента в NPS-опросе;
* `nps_group` — поле хранит информацию о том, к какой группе относится оценка клиента в опросе.

In [6]:
# сохраним DataFrame в csv-файл
df.to_csv('telecomm_csi_tableau.csv', index=False)

## [Ссылка на дашборды и презентацию на сайте Tableau Public](https://public.tableau.com/views/_16927819625830/sheet16?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link)